# Run spec2 pipeline on rate files

## Imports

In [ ]:
# Packages that allow us to get information about objects:
import asdf
import os
# Update to a path in your system (see details below at "Reference files")
os.environ["CRDS_PATH"] = "~/crds_cache_may2023/"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"


In [ ]:
# The calwebb_spec and spec3 pipelines
from jwst.pipeline import Spec2Pipeline
from jwst.pipeline import Spec3Pipeline

# individual steps
from jwst.assign_wcs import AssignWcsStep
from jwst.assign_wcs import nirspec
from jwst.background import BackgroundStep
from jwst.imprint import ImprintStep
from jwst.msaflagopen import MSAFlagOpenStep
from jwst.extract_2d import Extract2dStep
from jwst.srctype import SourceTypeStep
from jwst.wavecorr import WavecorrStep
from jwst.flatfield import FlatFieldStep
from jwst.pathloss import PathLossStep
from jwst.photom import PhotomStep
from jwst.cube_build import CubeBuildStep
from jwst.extract_1d import Extract1dStep
from jwst.combine_1d import Combine1dStep

# data models
from jwst import datamodels

# associations
from jwst.associations import asn_from_list
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base

# astropy
from astropy.io import fits
from astropy.utils.data import download_file
import astropy.units as u
from astropy import wcs
from astropy.wcs import WCS
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch, LinearStretch, AsinhStretch

from matplotlib import pyplot as plt

In [ ]:
import jwst
print('jwst',jwst.__version__)

## Files

In [ ]:
association_level2 = './jw02079-o004_f115w_f150wcat_asn.json'
rate = './jw02079004001_05101_00001_nis_rate.fits'

#association_level2 = './jw02079-o004_f150w_f150wcat_asn.json'
#rate = './jw02079004002_11101_00001_nis_rate.fits'

#association_level2 = './jw02079-o004_f200w_f150wcat_asn.json'
#rate = './jw02079004003_05101_00001_nis_rate.fits'


## Run spec2

In [ ]:
output_dir = './pipeline_output/'
# Create an instance of the pipeline class
spec2 = Spec2Pipeline()

# Set some parameters that pertain to the entire pipeline
#spec2.save_results = True
#spec2.output_dir = output_dir

# Here you could some parameters that pertain to some of the individual steps
#spec2.extract_2d.mmag_extract = 20 # Minimum magnitude of objects to extract for WFSS data
#spec2.extract_1d.bkg_fit = 'poly' # Fit a polynomial to the background values for each column or row
#spec2.bkg_subtract.skip = True # Skip the default image-from-image background subtraction method

# Call the call() method, using an input association file.
# Call method reinitialized the default values, so parameters need to be changed inside the call
result = spec2.call(association_level2, 
                    steps={'bkg_subtract':{'skip':True},
                           'extract_2d':{'wfss_nbright':500,
                                         'wfss_extract_half_height': 25,
                                         },
                           'photom':{'skip':True},
                          },
                    save_results=True,
                    output_dir=output_dir)


In [ ]:
calfile = './pipeline_output/jw02079004001_05101_00001_nis_cal.fits'
calimage = fits.open(calfile)

In [ ]:
source = 4
print(calimage['SCI',source].header['SOURCEID'])

In [ ]:
data = calimage['SCI',source].data
plt.imshow(data, origin='lower', vmin=0.4, vmax=0.8)
plt.colorbar()

There is no background subtraction so this is absolutely useless.

In [ ]:
x1dfile = './pipeline_output/jw02079004002_11101_00001_nis_x1d.fits'
x1dspec = fits.open(x1dfile)

In [ ]:
data = x1dspec['EXTRACT1D',source].data
plt.plot(data['WAVELENGTH'], data['FLUX'])
plt.xlabel('wavelength (um)')
plt.ylabel('flux (adu)')
plt.show()